In [2]:
from diffusers import StableDiffusionImg2ImgPipeline
import torch
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import cv2

# Charger une image
def load_image(path, size=512):
    img = Image.open(path).convert("RGB")
    img = img.resize((size, size))
    return img

In [ ]:
import sys
import torch
sys.path.append("../../facefusion/facefusion")

# Initialiser le pipeline
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [12]:
# Paramètres
prompt = "a realistic photo of a person"  # Le prompt peut être générique
strength = 0.05  # Très faible pour ne PAS altérer le contenu
guidance_scale = 1.0  # Faible aussi pour limiter le changement

origin_image = load_image("../../extracted_frames/alexandre.jpg")

with torch.no_grad():
    result1 = pipe(
        prompt=prompt,
        image=origin_image,
        strength=strength,
        guidance_scale=guidance_scale,
        num_inference_steps=5000
    ).images[0]

transf_image = load_image("../../extracted_frames_output/alexandre_florian_simswap_256_gfpgan_1.4_75_0.7.jpg")

with torch.no_grad():
    result2 = pipe(
        prompt=prompt,
        image=transf_image,
        strength=strength,
        guidance_scale=guidance_scale,
        num_inference_steps=5000
    ).images[0]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

In [13]:
import lpips
import torchvision.transforms as T

print("alex")
loss_fn = lpips.LPIPS(net='alex')  # ou 'vgg'

transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor()
])

img1 = transform(origin_image).unsqueeze(0)
img2 = transform(result1).unsqueeze(0)
score = loss_fn(img1, img2)
print(f"LPIPS score: {score.item()}")

img1 = transform(transf_image).unsqueeze(0)
img2 = transform(result2).unsqueeze(0)
score = loss_fn(img1, img2)
print(f"LPIPS score: {score.item()}")

print("vgg")
loss_fn = lpips.LPIPS(net='vgg')  # ou 'vgg'

transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor()
])

img1 = transform(origin_image).unsqueeze(0)
img2 = transform(result1).unsqueeze(0)
score = loss_fn(img1, img2)
print(f"LPIPS score: {score.item()}")

img1 = transform(transf_image).unsqueeze(0)
img2 = transform(result2).unsqueeze(0)
score = loss_fn(img1, img2)
print(f"LPIPS score: {score.item()}")

alex
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/alex/miniconda3/envs/env_sbi/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
LPIPS score: 0.004460480529814959
LPIPS score: 0.0039420113898813725
vgg
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /home/alex/miniconda3/envs/env_sbi/lib/python3.10/site-packages/lpips/weights/v0.1/vgg.pth
LPIPS score: 0.017657753080129623
LPIPS score: 0.016568021848797798
